# PS4: Let's build a Personal Shopper
In this problem set, we'll build a simple personal shopper program that helps users find items they might want to buy based on their preferences. The program will allow users to input their preferences and then suggest items from a predefined list that maximize their satisfaction (measured using a utility function) based on those preferences, subjet to a budget constraint.

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [49]:
include("Include.jl"); # This will load necessary packages and functions

First, let's build the `world(...)` function. 
* The `world(...)` function takes the $m$-dimensional action vector `a::Array{Int64,1}` where the elements of `a::Array{Int64,1}` are the indexes of the goods chosen from each categories, the amount of each good selected from each category from our agent is in the `n::Dict{Int,Array{Float64,1}}` dictionary, and returns the reward (utility) $r\sim\mathcal{D}_{a}$. associated with selecting this action. 

In [50]:
function world(a::Vector{Int64}, n::Array{Float64,1}, context::MyBanditConsumerContextModel)::Float64

    # initialize -
    γ = context.γ; # consumer preferences (unknown to bandits)
    σ = context.σ; # noise in utility calculation (unknown to bandits)
    B = context.B; # max budget (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)
    λ = context.λ; # sensitivity to the budget
    Z = context.Z; # noise model
    number_of_goods = context.m; # number of categories

    # compute the reward for this choice -
    Ū = 1.0;
    BC = 0.0;
    for i ∈ 1:number_of_goods
        
        # what action is being taken in this category?
        aᵢ = a[i]; # this is which good to purchase in category i -
        if (aᵢ == 0)
            # if aᵢ is 0, it means no good was chosen in this category, 
            # hence we should skip this category in the utility calculation
            continue; 
        end

        nᵢ = n[i]; # this is the quantity purchased of good aᵢ in category i
        Cᵢ = C[i]; # cost of chosen good in category i
        γᵢ = γ[i]; # preference of good in category i
        σᵢ = σ[i]; # standard dev for good i
   
        # update the utility -
        Ū += γᵢ*(nᵢ + σᵢ*rand(Z)); # compute the utility for this good, with noise
        
        # constraints and noise 
        BC += nᵢ*Cᵢ; # compute the budget constraint -
    end

    # compute the budget constraint violation -
    U = Ū - λ*max(0.0, (BC - B))^2; # use a penalty method to capture budget constraint

    # return the reward -
    return U;
end;

Next, let's build an algorithm model that we'll use to reason about the world, i.e., a model of our agent's decision making process.  we've modified the $\epsilon$-greedy algorithm to work with our contextual category-based bandit problem. The algorithm will now take into account the context provided by [the `MyBanditConsumerContextModel` model](src/Types.jl) and category-based actions when selecting actions.

#### Epsilon-Greedy with Categories Algorithm
In the _epsilon-greedy_ algorithm, the agent chooses the best arm with probability $1-\epsilon$ and a random arm with probability $\epsilon$. This approach balances exploration and exploitation by allowing the agent to explore different arms while also exploiting the best-known arm based on past rewards. The parameter $\epsilon$ controls the exploration rate: a higher value means more exploration, while a lower value means more exploitation.

* While [Slivkins](https://arxiv.org/abs/1904.07272) doesn't give a reference for the $\epsilon$-greedy algorithm, other sources point to (at least in part) to [Thompson and Thompson sampling, proposed in 1933 in the context of drug trials](https://arxiv.org/abs/1707.02038). Thus, the $\epsilon$-greedy algorithm, considered a classic algorithm in the multi-armed bandit literature. The algorithm is simple yet effective, making it a popular choice for many practical applications.

The agent has $K$ arms (choices), $\mathcal{A} = \left\{1,2,\dots,K\right\}$, where each arm encosed a _binary vector_ which describes the _basket_ of goods chosen by the agent, 
and the total number of rounds is $T$. The agent uses the following algorithm to choose which arms to pull (which action to take) during each round:

For $t = 1,2,\dots,T$:
1. _Initialize_: Roll a random number $p\in\left[0,1\right]$ and compute a threshold $\epsilon_{t}={t^{-1/3}}\cdot\left(K\cdot\log(t)\right)^{1/3}$.
2. _Exploration_: If $p\leq\epsilon_{t}$, choose a random $a_{t}\in\mathcal{A}$. Execute the action $a_{t}$ and receive a reward $r_{t}$ from the _adversary_ (nature). 
3. _Exploitation_: Else if $p>\epsilon_{t}$, choose action $a^{\star}$, the action with the highest average reward so far. Execute the action $a^{\star}_{t}$ and recieve a reward $r_{t}$ from the _adversary_ (nature).
4. Update list of rewards for $a_{t}\in\mathcal{A}$

Let's build [a `MyEpsilonGreedyAlgorithmModel` instance](src/Types.jl) which encapsulates the $\epsilon$-greedy logic that has been modified to work with categories. 
* __TODO__: To contstruct this type, we [use a custom `build(...)` method](src/Factory.jl) that takes the arms `K::Int64` and the items `n::Array{Float64,1}` array and returns an algorithmn model in the `algorithm::MyEpsilonGreedyAlgorithmModel` variable. In this simulationm, let `K = 12` and `n` to be array of ones.

In [51]:
algorithm = let

    # initialize -
    algorithm = nothing; # initialize the algorithm variable to nothing, this variable will be used to store the algorithm model
    K = 12; # suppose we have 12 possible goods that we can choose from
    n = ones(Float64, K); # for now, let's assume that we purchase a single unit of each good in each category (we can change this later)
    
    # TDOD: Build an algorithm model by by uncommenting the code block below
    algorithm = build(MyEpsilonGreedyAlgorithmModel, (
        K = K, # arms 
        n = n, # items dictionary
    ));

    # return the algorithm -
    algorithm;
end;

__Constants__: Set constants we'll use in the subsequent tasks. See the comment beside the value for a description of what it is, its permissible values, etc.

In [52]:
T = 2^14; # number of rounds for each decision task
B = 100.0; # Budget for shopper

## Task 2: Build the Context Models
In this task, we will build several models of the contextual information that will be used to inform the agent's recomendations. These models, which are [instances of the `MyBanditConsumerContextModel` type](src/Types.jl) hold various parameters that will be used in the `world(...)` funtion that we developed above. 
* _Hmmm. Why use a different model for contextual data_? We use a separate model for the contextual information because it allows us to encapsulate all the relevant parameters and settings in one place. This makes it easier to manage and modify the parameters as needed, without having to change the core logic of the `world(...)` function. Additionally, it allows us to easily pass around context models to other parts of our codebase that may need it.
* _What does this represent_? The contextual information in the `MyBanditConsumerContextModel` represents the parameters that will be used to score the utility of the goods chosen by the agent. This includes user sentiment parameters, budget constraints, and other relevant information that will help the agent make informed decisions about which goods to recommend.

Let's build the following contextual models:
* __Case 1: Unlimited budget, uniform positive sentiment__: This model assumes that the consumer has an unlimited budget ($\lambda = 0$) and uniform positive sentiment across all goods. This means that the consumer is equally likely to choose any good in each category, and there are no constraints on the amount of each good that can be selected. 
* __Case 2: Limited budget, positive sentiment__: This model assumes that the consumer has a limited budget $\lambda>0$ and positive sentiment (but not neccesarily uniform) towards goods. This means that the consumer is more likely to choose goods that they have a positive sentiment towards, and there are constraints on the amount of each good that can be selected based on the budget.
* __Case 3: Limited budget, mixed sentiment__: This model assumes that the consumer has a limited budget $\lambda>0$ and mixed sentiment towards goods. This means that some goods may have positive sentiment (i.e., they are preferred), while others may have negative sentiment (i.e., they are not preferred). The agent must balance the positive and negative sentiments when making recommendations, and there are constraints on the amount of each good that can be selected based on the budget.

Let's start with __case 1__. We save this case in the `simple_no_budget_context::MyBanditConsumerContextModel` variable.

In [53]:
simple_no_budget_context = let

    # initialize -
    context = nothing; # initialize the context variable to nothing, this variable will be used to store the context model
    K = algorithm.K; # number of arms in the algorithm, this should match the number of goods in the context model
    γ = Array{Float64,1}(undef, K); # consumer preferences (unknown to bandits)
    σ = Array{Float64,1}(undef, K); # noise in utility calculation (unknown to bandits)
    C = Array{Float64,1}(undef, K); # unit costs of goods (unknown to bandits)
    Z = Normal(0,1); # use a standard normal distribution for the noise model, this can be changed to any distribution as required
    λ = 0.0; # sensitivity to the budget constraint λ ≥ 0. If zero, then no penalty for budget constraint violation.

    # set the parameters -
    # preferences: If all γ[i] are equal to 1.0, then the bandit will be indifferent to the goods in each category.
    for i ∈ 1:K
        # Assigning values for γ, σ, and C for each good in the context model
        # For simplicity, let's assume we have K goods with equal preference
        # This can be customized as per the requirement of the simulation
        γ[i] = 1.0; # uniform preference for all goods
        σ[i] = 0.1; # uniform uncertainty for all goods, this can be adjusted based on the specific needs of the simulation
        C[i] = 10.0 + 10.0 * (i - 1); # linearly increasing costs for goods, this can be customized as per the requirement
    end

    # build a context model with the reqired parameters -
    context = build(MyBanditConsumerContextModel, (
        γ = γ, # consumer preferences (unknown to bandits)
        σ = σ, # noise in utility calculation (unknown to bandits)
        B = B, # max budget (unknown to bandits)
        C = C, # unit costs of goods (unknown to bandits)
        λ = λ, # sensitivity to the budget
        Z = Z, # noise model
        m = K, # number of categories (this should match the number of arms in the algorithm)
    )); # build the context

    # return 
    context;
end;

Next, build __case 2__. In this scenario, the consumer has a limited budget and positive sentiment. We set a budget constraint and define the user sentiment parameters to reflect positive sentiment towards all goods. We save this case in the `simple_with_budget_context::MyBanditConsumerContextModel` variable.`

In [54]:
simple_with_budget_context = let

    # initialize -
    context = nothing; # initialize the context variable to nothing, this variable will be used to store the context model
    K = algorithm.K; # number of arms in the algorithm, this should match the number of goods in the context model
    γ = Array{Float64,1}(undef, K); # consumer preferences (unknown to bandits)
    σ = Array{Float64,1}(undef, K); # noise in utility calculation (unknown to bandits)
    C = Array{Float64,1}(undef, K); # unit costs of goods (unknown to bandits)
    Z = Normal(0,1); # use a standard normal distribution for the noise model, this can be changed to any distribution as required
    λ = 10000.0; # sensitivity to the budget constraint λ ≥ 0. If zero, then no penalty for budget constraint violation.

    # set the parameters -
    # preferences: If all γ[i] are equal to 1.0, then the bandit will be indifferent to the goods in each category.
    for i ∈ 1:K
        # Assigning values for γ, σ, and C for each good in the context model
        # For simplicity, let's assume we have K goods with equal preference
        # This can be customized as per the requirement of the simulation
        γ[i] = 1.0; # uniform preference for all goods
        σ[i] = 0.1; # uniform uncertainty for all goods, this can be adjusted based on the specific needs of the simulation
        C[i] = 10.0 + 10.0 * (i - 1); # linearly increasing costs for goods, this can be customized as per the requirement
    end

    # build a context model with the reqired parameters -
    context = build(MyBanditConsumerContextModel, (
        γ = γ, # consumer preferences (unknown to bandits)
        σ = σ, # noise in utility calculation (unknown to bandits)
        B = B, # max budget (unknown to bandits)
        C = C, # unit costs of goods (unknown to bandits)
        λ = λ, # sensitivity to the budget
        Z = Z, # noise model
        m = K, # number of categories (this should match the number of arms in the algorithm)
    )); # build the context

    # return 
    context;
end;

Finally, build __case 3__. In this scenario, the consumer has a limited budget and mixed sentiment toward the selection of possible goods. We define both positive and negative user sentiment parameters to reflect the mixed sentiment towards goods, and set $\lambda > 0$.  We save this case in the `mixed_with_budget_context::MyBanditConsumerContextModel` variable.

In [55]:
mixed_with_budget_context = let

    # initialize -
    context = nothing; # initialize the context variable to nothing, this variable will be used to store the context model
    K = algorithm.K; # number of arms in the algorithm, this should match the number of goods in the context model
    γ = Array{Float64,1}(undef, K); # consumer preferences (unknown to bandits)
    σ = Array{Float64,1}(undef, K); # noise in utility calculation (unknown to bandits)
    C = Array{Float64,1}(undef, K); # unit costs of goods (unknown to bandits)
    Z = Normal(0,1); # use a standard normal distribution for the noise model, this can be changed to any distribution as required
    λ = 10000.0; # sensitivity to the budget constraint λ ≥ 0. If zero, then no penalty for budget constraint violation.

    # set the parameters -
    # preferences: If all γ[i] are equal to 1.0, then the bandit will be indifferent to the goods in each category.
    for i ∈ 1:K
        # Assigning values for γ, σ, and C for each good in the context model
        # For simplicity, let's assume we have K goods with equal preference
        # This can be customized as per the requirement of the simulation
        
        if (iseven(i) == true)
            γ[i] = 1.0; # positive preference for even indexed goods
        else
            γ[i] = -10.0; # negative preference for odd indexed goods
        end
        
        σ[i] = 0.1; # uniform uncertainty for all goods, this can be adjusted based on the specific needs of the simulation
        C[i] = 10.0 + 10.0 * (i - 1); # linearly increasing costs for goods, this can be customized as per the requirement
    end

    # change some preferences to create a mixed context 


    # build a context model with the reqired parameters -
    context = build(MyBanditConsumerContextModel, (
        γ = γ, # consumer preferences (unknown to bandits)
        σ = σ, # noise in utility calculation (unknown to bandits)
        B = B, # max budget (unknown to bandits)
        C = C, # unit costs of goods (unknown to bandits)
        λ = λ, # sensitivity to the budget
        Z = Z, # noise model
        m = K, # number of categories (this should match the number of arms in the algorithm)
    )); # build the context

    # return 
    context;
end;

## Task 3: Evaluation of Scenarios
In this task, we'll run different context models to evaluate how well our agent performs under different scenarios. This will help us understand how the choice of context model affects the agent's performance and the regret it incurs over time. In all cases, we will use the same agent and bandit algorithm but vary the context model to see how it influences the agent's decisions and performance. We'll use the $\epsilon$-greedy algorithm to estimate an optimal policy for the agent based on the contextual information provided by the `MyBanditConsumerContextModel` models.

We have already built all the models, so let's run all three scenarios below. In each case [we call the `solve(...)` method](src/Bandit.jl) to simulate the agent's decision-making process over a specified number of rounds, `T`, given the world defined by the `world(...)` function, and the context model. The [`solve(...)` method](src/Bandit.jl) returns the results of the simulation in the `results_case_*` variable.

In [56]:
results_case_1 = solve(algorithm, T = T, world = world, context=simple_no_budget_context); # compute allocation for case 1
results_case_2 = solve(algorithm, T = T, world = world, context=simple_with_budget_context); # compute allocation for case 2
results_case_3 = solve(algorithm, T = T, world = world, context=mixed_with_budget_context); # compute allocation for case 3

### Case 1: Unlimited Budget, Uniform Positive Sentiment
In this case, we will evaluate the agent's performance using the `simple_no_budget_context` model. This model assumes an unlimited budget and uniform positive sentiment across all goods.

In [57]:
table(results_case_1, algorithm, simple_no_budget_context) |> df -> pretty_table(df, tf = tf_simple)

======== ========== ============
   good   purchase   cumreward 
  Int64     String     Float64 
======== ========== ============
      1        Yes         1.0
      2        Yes         1.0
      3        Yes         1.0
      4        Yes         1.0
      5        Yes         1.0
      6        Yes         1.0
      7        Yes         1.0
      8        Yes         1.0
      9        Yes         1.0
     10        Yes         1.0
     11        Yes         1.0
     12        Yes         1.0
======== ========== ============


How much did our agent spend, and how much benefit was gained? `Unhide` the code block below to see the results of the agent's performance under this scenario. 

In [58]:
let

    # initialiize -
    results = results_case_1; # use results from case 1 for this example
    context = simple_no_budget_context;
    K = algorithm.K; # number of categories
    n = algorithm.n; # recommended number of items to purchase in each category
    γ = context.γ; # user preference for each good (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)

    # compute the best collection of goods -
    K = algorithm.K; # number of arms in the algorithm
    N = 2^K; # number of possible goods combinations (2^K) - this is the total number of combinations of goods we can have 

    μ = zeros(Float64, N); # average reward for each possible goods combination
    for a ∈ 1:N
        μ[a] = filter(x -> x != 0.0, results[:,a]) |> x-> mean(x)

        # fix NaN -
        if (isnan(μ[a]) == true)
            μ[a] = -Inf; # replace NaN with a big negative
        end
    end
    î = argmax(μ); # compute the arm with best average reward
    aₜ = digits(î, base=2, pad=K); # which goods do we select?

    U = Array{Float64,1}(undef, K); # initialize the array to store the goods selected
    for i ∈ 1:K
       U[i] = aₜ[i]*n[i]*γ[i]; # store the goods selected in the array
    end

    spend = Array{Float64,1}(undef, K); # initialize the array to store the spend on each good
    for i ∈ 1:K
        # calculate the spend on each good based on the recommended quantity and unit cost
        spend[i] = aₜ[i]*n[i] * C[i]; # total spend on each good
    end

    S̄ = sum(spend); # total spend for case 2
    Ū = sum(U); # total utility for case 2
    println("Case 1: Agent spent: $(S̄) USD and rcvd $(Ū) utils. ROI: $(Ū/S̄) util/USD"); # print total spend for case 1
    
end;

Case 1: Agent spent: 780.0 USD and rcvd 12.0 utils. ROI: 0.015384615384615385 util/USD


### Case 2: Limited Budget, Uniform Positive Sentiment
In this case, we will evaluate the agent's performance using the `simple_with_budget_context` model. This model assumes a limited budget and uniform positive sentiment across all goods.

In [59]:
table(results_case_2, algorithm, simple_with_budget_context) |> df -> pretty_table(df, tf = tf_simple) 

======== ========== ============
   good   purchase   cumreward 
  Int64     String     Float64 
======== ========== ============
      1        Yes         1.0
      2        Yes         1.0
      3        Yes         1.0
      4        Yes         1.0
      5         No         0.0
      6         No         0.0
      7         No         0.0
      8         No         0.0
      9         No         0.0
     10         No         0.0
     11         No         0.0
     12         No         0.0
======== ========== ============


How much did our agent spend, and how much benefit was gained? `Unhide` the code block below to see the results of the agent's performance under this scenario. 

In [60]:
let

    # initialiize -
    results = results_case_2; # use results from case 1 for this example
    context = simple_no_budget_context;
    K = algorithm.K; # number of categories
    n = algorithm.n; # recommended number of items to purchase in each category
    γ = context.γ; # user preference for each good (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)

    # compute the best collection of goods -
    K = algorithm.K; # number of arms in the algorithm
    N = 2^K; # number of possible goods combinations (2^K) - this is the total number of combinations of goods we can have 

    μ = zeros(Float64, N); # average reward for each possible goods combination
    for a ∈ 1:N
        μ[a] = filter(x -> x != 0.0, results[:,a]) |> x-> mean(x)

        # fix NaN -
        if (isnan(μ[a]) == true)
            μ[a] = -Inf; # replace NaN with a big negative
        end
    end
    î = argmax(μ); # compute the arm with best average reward
    aₜ = digits(î, base=2, pad=K); # which goods do we select?

    U = Array{Float64,1}(undef, K); # initialize the array to store the goods selected
    for i ∈ 1:K
       U[i] = aₜ[i]*n[i]*γ[i]; # store the goods selected in the array
    end

    spend = Array{Float64,1}(undef, K); # initialize the array to store the spend on each good
    for i ∈ 1:K
        # calculate the spend on each good based on the recommended quantity and unit cost
        spend[i] = aₜ[i]*n[i] * C[i]; # total spend on each good
    end

    S̄ = sum(spend); # total spend for case 2
    Ū = sum(U); # total utility for case 2
    println("Case 2: Agent spent: $(S̄) USD and rcvd $(Ū) utils. ROI: $(Ū/S̄) util/USD"); # print total spend for case 2
end;

Case 2: Agent spent: 100.0 USD and rcvd 4.0 utils. ROI: 0.04 util/USD


### Case 3: Limited Budget, Mixed Sentiment
In this case, we will evaluate the agent's performance using the `mixed_with_budget_contextt` model. This model assumes a limited budget and mixed positive and negative sentiment across all goods.

In [61]:
table(results_case_3, algorithm, mixed_with_budget_context) |> df -> pretty_table(df, tf = tf_simple) 

======== ========== ============
   good   purchase   cumreward 
  Int64     String     Float64 
======== ========== ============
      1         No        -0.0
      2        Yes         1.0
      3         No        -0.0
      4        Yes         1.0
      5         No        -0.0
      6         No         0.0
      7         No        -0.0
      8         No         0.0
      9         No        -0.0
     10         No         0.0
     11         No        -0.0
     12         No         0.0
======== ========== ============


In [62]:
let

    # initialiize -
    results = results_case_3; # use results from case 1 for this example
    context = simple_no_budget_context;
    K = algorithm.K; # number of categories
    n = algorithm.n; # recommended number of items to purchase in each category
    γ = context.γ; # user preference for each good (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)

    # compute the best collection of goods -
    K = algorithm.K; # number of arms in the algorithm
    N = 2^K; # number of possible goods combinations (2^K) - this is the total number of combinations of goods we can have 

    μ = zeros(Float64, N); # average reward for each possible goods combination
    for a ∈ 1:N
        μ[a] = filter(x -> x != 0.0, results[:,a]) |> x-> mean(x)

        # fix NaN -
        if (isnan(μ[a]) == true)
            μ[a] = -Inf; # replace NaN with a big negative
        end
    end
    î = argmax(μ); # compute the arm with best average reward
    aₜ = digits(î, base=2, pad=K); # which goods do we select?

    U = Array{Float64,1}(undef, K); # initialize the array to store the goods selected
    for i ∈ 1:K
       U[i] = aₜ[i]*n[i]*γ[i]; # store the goods selected in the array
    end

    spend = Array{Float64,1}(undef, K); # initialize the array to store the spend on each good
    for i ∈ 1:K
        # calculate the spend on each good based on the recommended quantity and unit cost
        spend[i] = aₜ[i]*n[i] * C[i]; # total spend on each good
    end

    S̄ = sum(spend); # total spend for case 2
    Ū = sum(U); # total utility for case 2
    println("Case 3: Agent spent: $(S̄) USD and rcvd $(Ū) utils. ROI: $(Ū/S̄) util/USD"); # print total spend for case 2
end;

Case 3: Agent spent: 60.0 USD and rcvd 2.0 utils. ROI: 0.03333333333333333 util/USD


## Tests
In the code block below, we check some values in your notebook and give you feedback on which items are correct or different. `Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing.

In [63]:
let 
    @testset verbose = true "CHEME 5820 Problem Set 4 Test Suite" begin

        @testset "Task 1: Setup, Prerequisites and Data" begin
            @test _DID_INCLUDE_FILE_GET_CALLED == true
            @test isdefined(Main, :world) == true
            @test isnothing(algorithm) == false
        end

        @testset "Task 2: Context models" begin
            @test isnothing(simple_no_budget_context) == false # Test for simple no budget context
            @test isnothing(simple_with_budget_context) == false # Test for simple with budget context
            @test isnothing(mixed_with_budget_context) == false # Test for mixed context with budget
            @test simple_no_budget_context.λ == 0; #
        end

    end
end;

Test Summary:                           | Pass  Total  Time
CHEME 5820 Problem Set 4 Test Suite     |    7      7  0.0s
  Task 1: Setup, Prerequisites and Data |    3      3  0.0s
  Task 2: Context models                |    4      4  0.0s
